# Iodine Spectrometer Data Collection

In [ ]:
import time
from time import perf_counter, sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pydaqmx_helper.adc import ADC

### Variable Naming

In [ ]:
sample_rate = 5000  # samples per second
bin_size = 0.05  # seconds per bin
countdown = 5  # seconds to wait before starting acquisition

In [ ]:
stop = False
def _enter_to_stop():
    global stop
    input("Press Enter to stop...\n")
    stop = True

In [ ]:
for s in range(countdown, 0, -1): # counting down so i know when to start the motor
    print(s)
    sleep(1)
print("Starting acquisition...")


In [ ]:
loop_stop = 0

myADC = ADC()
myADC.addChannels([0], minRange = -10, maxRange = 10)
data = myADC.sampleVoltages(sample_rate * bin_si

while loop_stop == 0:



In [ ]:
myADC = ADC()
myADC.addChannels([0], minRange = -10, maxRange = 10)


In [ ]:
import time
import numpy as np
import pandas as pd
from pydaqmx_helper.adc import ADC

# --- User settings ---
CHANNEL = 0
V_RANGE = (-10, 10)
SAMPLE_RATE = 5000
BIN_SIZE = 0.05  # seconds
DURATION = float(input("Scan duration (s): ") or "120")
LABEL = input("Label: ") or "run"

# --- Press Enter to stop ---
stop = False
def _enter_to_stop():
    global stop
    input("Press Enter to stop...\n")
    stop = True

import threading
threading.Thread(target=_enter_to_stop, daemon=True).start()

# --- Countdown ---
for s in range(5, 0, -1):
    print(s)
    time.sleep(1)
print("Starting acquisition...")

adc = ADC()
adc.addChannels([CHANNEL], minRange=V_RANGE[0], maxRange=V_RANGE[1])

samples_per_bin = int(SAMPLE_RATE * BIN_SIZE)
n_bins = int(DURATION / BIN_SIZE)
results = []

start = time.perf_counter()
while len(results) < n_bins and not stop:
    data = adc.sampleVoltages(samples_per_bin, SAMPLE_RATE)
    if np.ndim(data) > 1:
        data = data[:, 0]
    t = time.perf_counter() - start
    results.append([t, np.mean(data), np.std(data)])

df = pd.DataFrame(results, columns=["time_s", "mean_V", "std_V"])
fname = f"{LABEL}_{int(time.time())}.csv"
df.to_csv(fname, index=False)
print(f"Saved: {fname}")